# Warm up
Let's start with some warm up exercises to get you familiar with the database and do some SQL querying.
Make sure to have downloaded the NBA dataset form Kaggle as described in the README.

In [ ]:
#!pip install -e .

In [ ]:
import sqlite3 as sql
import pandas as pd
import seaborn
from pathlib import Path
import matplotlib.pyplot as plt
from nba.data_processing import col_all_table, get_pts_team_year, get_team_attendance, get_wingspan, get_player_school
from nba.visualization import plot_avg_points_led, plot_avg_attend, box_plot

## TODO: Use pathlib to get the path to the data directory. Path(__file__) won't work in a Jupyter notebook.
DATA_PATH = Path.cwd() / 'data/nba.sqlite' 

## Create a SQLite connection and run some SQL queries

In [ ]:
con = sql.connect(DATA_PATH)  # connect to the database
query = 'SELECT * FROM game LIMIT 10'  # write a query that selects the first 10 rows from the game table
top_10_game = pd.read_sql(query, con)  # read the query into a pandas dataframe
top_10_game.head()  # print the dataframe

In [ ]:
# let's look at the schema of the database
# TODO: sqlite_master is the name of the table you want to inspeact: Replace TABLE_NAME with the name of the table you want to inspect
query = """
    SELECT * 
    FROM sqlite_master
"""
pd.read_sql(query, con).sort_values('name')

In [ ]:
# let's see all the tables we have in the database
# TODO: Filter for tables only
query = """
    SELECT name
    FROM sqlite_master
    WHERE type = "table"
"""
pd.read_sql(query, con)

In [ ]:
# let's have a look at the columns in the game table
query = """
    PRAGMA table_info(draft_combine_stats)
"""
pd.read_sql(query, con)

In [ ]:
# let's find the earliest recorded game
query = """
    SELECT game_date
    FROM game
    ORDER BY game_date
    LIMIT 1
"""

print(pd.read_sql(query, con))

# let's find the latest recorded game
query = """
    SELECT game_date
    FROM game
    ORDER BY game_date DESC
    LIMIT 1
"""

print(pd.read_sql(query, con))


In [ ]:
# look at column names of all tables
col_all_table(con)

In [ ]:
### Insight1
#From 2015 to 2017, Golden State Warriors (GSW) dominated with the highest average points for three consectuive years. 
#This overlapped with the Golden State Warriors' dynasty years. Staring in 2018, other teams began to lead in average points. 
# Despite this shift in leadership, average points scored by leading teams have also increased over time, reflecting more intense and competitive games.
pts_team_year = get_pts_team_year(con)
plot_avg_points_led(pts_team_year)

In [ ]:
### Insight2
#The Chicago Bulls (CHI) have the highest average attendece at home games over the past two decades. 
#This can be attributed to their historical success and the large arena capacity. 
# The other teams (MIA, TOR, etc.) also have high attendece, with the majority balanced between 19k to 14k. 
# RMD and MLN had the lowest attendece, but even they exceeded 10k, illustarting the large and highly engaed US basketball game market.
team_attendance = get_team_attendance(con)
plot_avg_attend(team_attendance)

In [ ]:
### Insight3
# Get wingspan data for each season
data = get_wingspan(con)

# Create a box plot
box_plot(data)

In [ ]:
# Print summary statistics for each season
print("\nSummary Statistics by Season:")
print(data.groupby('season')['wingspan'].describe()) 

In [ ]:
### Insight4
# Find the school that produced the most NBA players using common_player_Info table
dataset = get_player_school(con)
seaborn.barplot(data=dataset, x='player_count', y='school')